# Data-driven Modeling
From wikipedia: "Data-driven models are a class of computational models that primarily rely on historical data collected throughout a system's or process' lifetime to establish relationships between input, internal, and output variables."

Data-driven models are - at their highest level - <i>any</i> model that uses data to inform predictions.

Contrast this with purely physics-based models that use hypotheses, (or maybe even just guesses) to relate inputs to outputs.

# Linear Regression
As an example of these two regimes, suppose we want to make predictions about a simple variable $y(x)$